# Multithreading & Concurrency

**Level 3: Advanced - Mastering Thread-Safe, High-Performance Applications**

**Master concurrent programming, synchronization, and thread-safe design for enterprise-scale applications**

---

## Thread Fundamentals

**Understanding thread lifecycle, creation patterns, and basic synchronization**

In [ ]:
// Thread creation and lifecycle demonstration
import java.util.concurrent.atomic.*;
import java.util.*;

public class ThreadFundamentals {

    // Thread with Runnable interface
    static class WorkerRunnable implements Runnable {
        private final String name;
        private final int id;

        public WorkerRunnable(String name, int id) {
            this.name = name;
            this.id = id;
        }

        @Override
        public void run() {
            String threadName = Thread.currentThread().getName();
            System.out.println(threadName + ": " + name + " (ID: " + id + ") starting work...\n");

            try {
                // Simulate work
                for (int i = 1; i <= 5; i++) {
                    Thread.sleep(100); // Simulate processing time
                    System.out.println(threadName + ": " + name + " processing step " + i);
                }

                System.out.println(threadName + ": " + name + " work completed!\n");
            } catch (InterruptedException e) {
                System.out.println(threadName + ": " + name + " was interrupted!");
                Thread.currentThread().interrupt(); // Restore interrupted status
            }
        }
    }

    // Thread extending Thread class
    static class WorkerThread extends Thread {
        private final String taskName;
        private final int priority;

        public WorkerThread(String taskName, int priority) {
            this.taskName = taskName;
            this.priority = priority;
            setName("Worker-" + taskName);
            setPriority(priority);
        }

        @Override
        public void run() {
            System.out.println(getName() + " started at: " + new Date());
            System.out.println(getName() + " priority: " + getPriority());

            try {
                Thread.sleep(50 * taskName.length()); // Different processing times
                System.out.println(getName() + " completed at: " + new Date() + "\n");
            } catch (InterruptedException e) {
                System.out.println(getName() + " interrupted!");
            }
        }
    }

    public static void demonstrateBasicThreading() {
        System.out.println("=== BASIC THREADING - Runnable vs Thread ===\n");

        System.out.println("1. Using Runnable interface:");
        Thread thread1 = new Thread(new WorkerRunnable("Database Backup", 1001));
        Thread thread2 = new Thread(new WorkerRunnable("File Processing", 1002));

        thread1.start(); // Start thread execution
        thread2.start();

        // Wait for both to complete
        try {
            thread1.join(); // Wait for thread1 to finish
            thread2.join(); // Wait for thread2 to finish
        } catch (InterruptedException e) {
            System.out.println("Main thread interrupted!");
        }

        System.out.println("2. Extending Thread class:");
        WorkerThread worker1 = new WorkerThread("Email", Thread.MAX_PRIORITY);
        WorkerThread worker2 = new WorkerThread("Report", Thread.NORM_PRIORITY);
        WorkerThread worker3 = new WorkerThread("Cleanup", Thread.MIN_PRIORITY);

        worker1.start();
        worker2.start();
        worker3.start();

        try {
            Thread.sleep(500); // Let threads start
        } catch (InterruptedException e) {
            // Handle interruption
        }

        System.out.println("\nActive threads: " + Thread.activeCount());
        System.out.println("Current thread: " + Thread.currentThread().getName());
    }

    public static void demonstrateThreadStates() {
        System.out.println("\n=== THREAD STATES LIFECYCLE ===\n");

        Thread.State[] states = Thread.State.values();
        for (Thread.State state : states) {
            System.out.println("Thread state: " + state);
            System.out.println("  Description: " + getStateDescription(state));
            System.out.println();
        }

        // Demonstrate state transitions
        System.out.println("Thread state transitions demonstration:");
        Thread demo = new Thread(() -> {
            try {
                System.out.println("Thread in RUNNABLE state");
                Thread.sleep(100);
                System.out.println("Thread in TIMED_WAITING state (sleeping)");
                Thread.sleep(100);
            } catch (InterruptedException e) {
                Thread.currentThread().interrupt();
            }
        });

        System.out.println("Before start: " + demo.getState());
        demo.start();

        try {
            Thread.sleep(50);
            System.out.println("After start: " + demo.getState());
            Thread.sleep(200);
            System.out.println("After sleep: " + demo.getState());
        } catch (InterruptedException e) {
            Thread.currentThread().interrupt();
        }

        System.out.println("\nThread lifecycle:");
        System.out.println("NEW → RUNNABLE → WAITING/TIMED_WAITING/BLOCKED → TERMINATED");
    }

    private static String getStateDescription(Thread.State state) {
        switch (state) {
            case NEW: return "Thread created but not yet started";
            case RUNNABLE: return "Thread is running or ready to run";
            case BLOCKED: return "Thread waiting for monitor lock";
            case WAITING: return "Thread waiting indefinitely for another thread";
            case TIMED_WAITING: return "Thread waiting for specified time";
            case TERMINATED: return "Thread has completed execution";
            default: return "Unknown state";
        }

    public static void demonstrateDaemonThreads() {
        System.out.println("\n=== DAEMON THREADS ===\n");

        Thread daemon = new Thread(() -> {
            while (true) {
                try {
                    Thread.sleep(1000);
                    System.out.println("Daemon thread running... (" + new Date() + ")");
                } catch (InterruptedException e) {
                    System.out.println("Daemon thread interrupted!");
                    break;
                }
            }
        }, "BackgroundMonitor");

        // Must be set BEFORE starting
        daemon.setDaemon(true);
        daemon.start();

        System.out.println("Main thread is daemon: " + Thread.currentThread().isDaemon());
        System.out.println("Background monitor is daemon: " + daemon.isDaemon());
        System.out.println("Main thread will exit in 3 seconds, daemon will terminate automatically...\n");

        try {
            Thread.sleep(3000);
        } catch (InterruptedException e) {
            Thread.currentThread().interrupt();
        }

        System.out.println("Main thread exiting - daemon thread will die automatically");
    }

    public static void main(String[] args) {
        demonstrateBasicThreading();
        demonstrateThreadStates();
        demonstrateDaemonThreads();

        System.out.println("\n🎯 THREAD FUNDAMENTALS MASTERED:");
        System.out.println("• Thread creation with Runnable vs extending Thread");
        System.out.println("• Thread lifecycle and state management");
        System.out.println("• Thread priorities and daemon threads");
        System.out.println("• Basic thread communication and coordination");
        System.out.println("• Exception handling in multi-threaded code");

        System.out.println("\nNext: Dive into synchronization and thread safety!");
    }
}


## Synchronization & Thread Safety

**Master synchronized blocks, locks, and atomic operations for thread-safe code**

In [ ]:
// Synchronization techniques for thread-safe programming
import java.util.concurrent.*;
import java.util.concurrent.locks.*;
import java.util.concurrent.atomic.*;
import java.util.*;

public class SynchronizationTechniques {

    // Race condition example - NOT thread safe
    static class UnsafeCounter {
        private int count = 0;

        public void increment() {
            count++; // Race condition: read-modify-write not atomic
        }

        public int getCount() {
            return count;
        }
    }

    // Thread-safe with synchronized method
    static class SafeCounter {
        private int count = 0;

        public synchronized void increment() {
            count++; // Now thread-safe
        }

        public synchronized int getCount() {
            return count;
        }
    }

    // Atomic operations with AtomicInteger
    static class AtomicCounter {
        private final AtomicInteger count = new AtomicInteger(0);

        public void increment() {
            count.incrementAndGet(); // Atomic operation
        }

        public int getCount() {
            return count.get(); // Atomic read
        }

        public boolean compareAndSet(int expected, int update) {
            return count.compareAndSet(expected, update);
        }
    }

    public static void demonstrateRaceCondition() {
        System.out.println("=== RACE CONDITION DEMONSTRATION ===\n");

        System.out.println("Test 1: Unsafe counter with race condition");
        UnsafeCounter unsafeCounter = new UnsafeCounter();
        testCounter(unsafeCounter, "Unsafe");

        System.out.println("\nTest 2: Safe counter with synchronized methods");
        SafeCounter safeCounter = new SafeCounter();
        testCounter(safeCounter, "Safe-Synchronized");

        System.out.println("\nTest 3: Safe counter with AtomicInteger");
        AtomicCounter atomicCounter = new AtomicCounter();
        testCounter(atomicCounter, "Safe-Atomic");

        System.out.println("\n🎯 RACE CONDITION LESSON:");
        System.out.println("When multiple threads modify shared data concurrently,");
        System.out.println("operations that are not atomic can produce incorrect results.");
        System.out.println("Use synchronization to protect shared state!");
    }

    private static void testCounter(Object counter, String type) {
        final int THREADS = 4;
        final int OPERATIONS_PER_THREAD = 10000;

        Runnable task = () -> {
            for (int i = 0; i < OPERATIONS_PER_THREAD; i++) {
                if (counter instanceof UnsafeCounter) {
                    ((UnsafeCounter) counter).increment();
                } else if (counter instanceof SafeCounter) {
                    ((SafeCounter) counter).increment();
                } else if (counter instanceof AtomicCounter) {
                    ((AtomicCounter) counter).increment();
                }
            }
        };

        // Start multiple threads
        Thread[] threads = new Thread[THREADS];
        for (int i = 0; i < THREADS; i++) {
            threads[i] = new Thread(task);
            threads[i].start();
        }

        // Wait for all to complete
        for (int i = 0; i < THREADS; i++) {
            try {
                threads[i].join();
            } catch (InterruptedException e) {
                Thread.currentThread().interrupt();
            }
        }

        // Check result
        int result = 0;
        if (counter instanceof UnsafeCounter) {
            result = ((UnsafeCounter) counter).getCount();
        } else if (counter instanceof SafeCounter) {
            result = ((SafeCounter) counter).getCount();
        } else if (counter instanceof AtomicCounter) {
            result = ((AtomicCounter) counter).getCount();
        }

        int expected = THREADS * OPERATIONS_PER_THREAD;
        System.out.println(type + " Counter:");
        System.out.println("  Expected: " + expected);
        System.out.println("  Actual:   " + result);
        System.out.println("  Correct:  " + (result == expected));
    }

    public static void demonstrateSynchronizationBlocks() {
        System.out.println("\n=== SYNCHRONIZED BLOCKS vs METHODS ===\n");

        BankAccount account = new BankAccount(1000);

        // Start multiple concurrent transfers
        Runnable transferTask = () -> {
            Random random = new Random();
            for (int i = 0; i < 100; i++) {
                int amount = random.nextInt(100) + 1;
                if (random.nextBoolean()) {
                    account.deposit(amount);
                } else {
                    account.withdraw(amount);
                }
            }
        };

        Thread[] threads = new Thread[5];
        for (int i = 0; i < threads.length; i++) {
            threads[i] = new Thread(transferTask, "Transaction-" + (i + 1));
            threads[i].start();
        }

        // Wait for completion
        for (Thread thread : threads) {
            try {
                thread.join();
            } catch (InterruptedException e) {
                Thread.currentThread().interrupt();
            }
        }

        System.out.println("Final balance: $" + account.getBalance());
        System.out.println("Account integrity preserved: " + account.checkIntegrity());
    }

    static class BankAccount {
        private int balance;
        private final Object lock = new Object(); // Separate lock object

        public BankAccount(int initialBalance) {
            this.balance = initialBalance;
        }

        // Synchronized method (locks on 'this')
        public synchronized void deposit_Method(int amount) {
            balance += amount;
        }

        // Synchronized block (more flexible)
        public void deposit(int amount) {
            synchronized (lock) {  // Can lock on different objects
                balance += amount;
                System.out.println(Thread.currentThread().getName() + " deposited $" + amount +
                                 " - New balance: $" + balance);
            }
        }

        public void withdraw(int amount) {
            synchronized (lock) {
                if (balance >= amount) {
                    balance -= amount;
                    System.out.println(Thread.currentThread().getName() + " withdrew $" + amount +
                                     " - New balance: $" + balance);
                } else {
                    System.out.println(Thread.currentThread().getName() + " failed to withdraw $" + amount +
                                     " (insufficient funds)");
                }
            }
        }

        public int getBalance() {
            synchronized (lock) {
                return balance;
            }
        }

        // For integrity checking
        public boolean checkIntegrity() {
            synchronized (lock) {
                return balance >= 0; // Simple integrity check
            }
        }
    }

    public static void demonstrateLocks() {
        System.out.println("\n=== REENTRANT LOCK DEMONSTRATION ===\n");

        SharedResource resource = new SharedResource();

        Runnable task = () -> {
            try {
                resource.methodA();
            } catch (InterruptedException e) {
                Thread.currentThread().interrupt();
            }
        };

        Thread thread1 = new Thread(task, "Reentrant-1");
        Thread thread2 = new Thread(task, "Reentrant-2");

        thread1.start();
        thread2.start();

        try {
            thread1.join();
            thread2.join();
        } catch (InterruptedException e) {
            Thread.currentThread().interrupt();
        }

        System.out.println("\nReentrantLock completed successfully!");
        System.out.println("Same thread can acquire the same lock multiple times.");
        System.out.println("Lock must be released the same number of times it was acquired.");
    }

    static class SharedResource {
        private final ReentrantLock lock = new ReentrantLock();
        private final AtomicInteger counter = new AtomicInteger(0);

        public void methodA() throws InterruptedException {
            lock.lock(); // Acquire lock
            try {
                System.out.println(Thread.currentThread().getName() +
                                 " acquired lock in methodA (hold count: " + lock.getHoldCount() + ")");
                Thread.sleep(50); // Simulate work
                methodB(); // Calls methodB (reentrant)
            } finally {
                lock.unlock(); // Always unlock in finally
                System.out.println(Thread.currentThread().getName() +
                                 " released lock from methodA");
            }
        }

        public void methodB() throws InterruptedException {
            lock.lock(); // Acquire lock again (reentrant)
            try {
                System.out.println(Thread.currentThread().getName() +
                                 " acquired lock in methodB (hold count: " + lock.getHoldCount() + ")");
                counter.incrementAndGet();
                int value = counter.get();
                System.out.println(Thread.currentThread().getName() + " incremented counter to " + value);
                Thread.sleep(50); // Simulate work
            } finally {
                lock.unlock(); // Must unlock
                System.out.println(Thread.currentThread().getName() +
                                 " released lock from methodB");
            }
        }
    }

    public static void main(String[] args) {
        demonstrateRaceCondition();
        demonstrateSynchronizationBlocks();
        demonstrateLocks();

        System.out.println("\n🎯 SYNCHRONIZATION MASTERED:");
        System.out.println("• Race conditions and thread safety issues");
        System.out.println("• synchronized methods vs blocks");
        System.out.println("• ReentrantLock for advanced synchronization");
        System.out.println("• Atomic classes for lock-free programming");
        System.out.println("• Choosing appropriate synchronization strategy");
    }
}
